# CDB GUI


In [ ]:
##### 设定cdb参数值 （只影响界面初值，GUI中可改） #####
# buffer缓冲区大小，取决于硬件及C代码
buffSize_init = "15 * 4 * 1024 / 2"
# 触发时间阈值（特定触发模式下才用到）
trigTimeTar_init = "40 * 2"
# buffer缓冲区起始地址
startAddr_init = "0x9000"
# dump大小
dumpSize_init = "15 * 4 * 1024 / 2"
# 主中断频率
IsrFreq_init = "30e3"
# fft频率显示范围
fftRange = (0,400)
# fft是否使用汉宁窗
fftUseHanningWin = True

##### 定义界面元素初值 #####

paraSetPreDefName = [
    "CH1_cur_mode",  # 1
    "",  # 2
    "",  # 3
    "",  # 4
    "",  # 5
    "",  # 6
]

trigSrcPreDefName = [
    "CH1_Udc",  # 1
    "",  # 2
]

logSrcPreDefName = [
    "CH1_Udc",  # 1
    "",  # 2
    "",  # 3
    "",  # 4
    "",  # 5
    "",  # 6
    "",  # 7
    "",  # 8
    "",  # 9
    "",  # 10
    "",  # 11
    "",  # 12
    "",  # 13
    "",  # 14
    "",  # 15
    "",  # 16
]

# 开启事件循环
%gui qt5
%matplotlib qt5

# 初始化库
%run -i cdb_gui_init.py


# 与此同时，部分脚本 API 仍然可用，但不要在 GUI 界面正在工作时运行脚本


In [ ]:
# 脚本接口示例

# 读变量
print(readMCUVarUtil("CH1_cur_mode"))

# 写变量
setMCUVarUtil("CH1_cur_mode", 0)

# 导出变量（type:{name:[numpyArrays, ...]}）
gExportDict

In [ ]:
# 变开关频率 （10-20k？） 烤电感
tar_E_freq = -20
pwm_freq = 20000
ctrl_freq = 1*pwm_freq
tar_thetaEInc = tar_E_freq/ctrl_freq

# setMCUVarUtil("CH1_cur_mode", 0)
# setMCUVarUtil("PWMFreqReqVal", pwm_freq)
# setMCUVarUtil("PWMFreqChgEn", 1)
setMCUVarUtil("thetaEInc", tar_thetaEInc)
# print(readMCUVarUtil("PWMFreqReqVal"))
print(readMCUVarUtil("thetaEInc"))
print(time.strftime('%H:%M:%S', time.localtime(time.time())))

In [ ]:
# 编码器零位自校正1
tar_E_freq = 10
ctrl_freq = 20000
tar_thetaEInc = tar_E_freq/ctrl_freq
Curr = 6

# 初始化
setMCUVarUtil("IProtectFlg_CH1", 0)
setMCUVarUtil("SPDProtectFlg", 0)

setMCUVarUtil("CH1_cur_mode", 0)
setMCUVarUtil("speed_mode", 0)

setMCUVarUtil("CH1_ext_fcn", 1)

# 先让电机转几圈
setMCUVarUtil("CH1_angle_mode", 0)
setMCUVarUtil("channel_mode", 1)
setMCUVarUtil("torque_mode", 0)
setMCUVarUtil("targetThetaE_CH1", 0)

setMCUVarUtil("thetaEInc", tar_thetaEInc)

setMCUVarUtil("CH1_cur_mode", 2)

setMCUVarUtil("targetId_CH1", Curr)

setMCUVarUtil("CH1_angle_mode", 1)

time.sleep(2)

# 再用预定位法
setMCUVarUtil("CH1_angle_mode", 0)
setMCUVarUtil("targetThetaE_CH1", 0)

temp = 0.3
delta = 0.01
while (temp > delta):
    setMCUVarUtil("targetThetaE_CH1", temp)
    temp -= delta
setMCUVarUtil("targetThetaE_CH1", 0)

time.sleep(1)
upper = readMCUVarUtil("thetaEnco_raw")
print(f"upper={upper}")

temp = 0.3
delta = 0.01
while (temp > delta):
    setMCUVarUtil("targetThetaE_CH1", (-temp) % 1)
    temp -= delta
setMCUVarUtil("targetThetaE_CH1", 0)

time.sleep(1)
lower = readMCUVarUtil("thetaEnco_raw")
print(f"lower={lower}")

setMCUVarUtil("CH1_cur_mode", 0)

In [ ]:
if (abs(upper-lower) > 10000):
    if (upper < lower):
        upper = upper+65536
    else:
        lower = lower+65536
mid = int((upper+lower)/2.0)
print(f"mid={mid}")
mid = int((upper+lower)/2.0+0.5) % 16384
print(f"mid2={mid}")
setMCUVarUtil("thetaEnco_raw_offset", mid)

In [ ]:
# 编码器零位自校正2
tar_E_freq = 10
ctrl_freq = 20000
tar_thetaEInc = tar_E_freq/ctrl_freq
Curr = 3

# 初始化
setMCUVarUtil("IProtectFlg_CH2", 0)
setMCUVarUtil("SPDProtectFlg", 0)

setMCUVarUtil("CH2_cur_mode", 0)
setMCUVarUtil("speed_mode2", 0)

setMCUVarUtil("CH2_ext_fcn", 1)

# 先让电机转几圈
setMCUVarUtil("CH2_angle_mode", 0)
setMCUVarUtil("channel_mode2", 1)
setMCUVarUtil("torque_mode2", 0)
setMCUVarUtil("targetThetaE_CH2", 0)

setMCUVarUtil("thetaEInc2", tar_thetaEInc)

setMCUVarUtil("CH2_cur_mode", 2)

setMCUVarUtil("targetId_CH2", Curr)

setMCUVarUtil("CH2_angle_mode", 1)

time.sleep(2)

# 再用预定位法
setMCUVarUtil("CH2_angle_mode", 0)
setMCUVarUtil("targetThetaE_CH2", 0)

temp = 0.3
delta = 0.01
while (temp > delta):
    setMCUVarUtil("targetThetaE_CH2", temp)
    temp -= delta
setMCUVarUtil("targetThetaE_CH2", 0)

time.sleep(1)
upper = readMCUVarUtil("thetaEnco_raw2")
print(f"upper={upper}")

temp = 0.3
delta = 0.01
while (temp > delta):
    setMCUVarUtil("targetThetaE_CH2", (-temp) % 1)
    temp -= delta
setMCUVarUtil("targetThetaE_CH2", 0)

time.sleep(1)
lower = readMCUVarUtil("thetaEnco_raw2")
print(f"lower={lower}")

setMCUVarUtil("CH2_cur_mode", 0)

In [ ]:
if (abs(upper-lower) > 10000):
    if (upper < lower):
        upper = upper+65536
    else:
        lower = lower+65536
mid = int((upper+lower)/2.0)
print(f"mid={mid}")
mid = int((upper+lower)/2.0+0.5) % 16384
print(f"mid2={mid}")
setMCUVarUtil("thetaEnco_raw_offset2", mid)

In [ ]:
# 手动取平均
temp = []
for i in range(200):
    temp.append(readMCUVarUtil("CH1_Utar.q"))

print(sum(temp)/len(temp))

In [ ]:
# 集中修改电流控制器PI参数

print(readMCUVarUtil("CH1_IdPI.kp"))
print(readMCUVarUtil("CH1_IdPI.ki"))
print(readMCUVarUtil("CH1_IqPI.kp"))
print(readMCUVarUtil("CH1_IqPI.ki"))

d_kp = 0.3
d_ki = 1000
q_kp = 0.3
q_ki = 1000

setMCUVarUtil("CH1_IdPI.kp", d_kp)
setMCUVarUtil("CH1_IdPI.ki", d_ki)
setMCUVarUtil("CH1_IqPI.kp", q_kp)
setMCUVarUtil("CH1_IqPI.ki", q_ki)

In [ ]:
# 集中修改电压控制器PI参数
Udc_kp = 733
Udc_ki = 2.44

Udc_kp = 700
Udc_ki = 2

setMCUVarUtil("UdcPI.kp", Udc_kp)
setMCUVarUtil("UdcPI.ki", Udc_ki)

In [ ]:
# 修改转速控制器PI参数

omega_kp = 0.0100
omega_ki = 0.004000

setMCUVarUtil("omegaPI.kp", omega_kp)
setMCUVarUtil("omegaPI.ki", omega_ki)
setMCUVarUtil("omegaPI.kb", omega_ki)

print(readMCUVarUtil("omegaPI.kp"))
print(readMCUVarUtil("omegaPI.ki"))
print(readMCUVarUtil("omegaPI.kb"))

In [ ]:
# prepos run prepare
setMCUVarUtil("IProtectFlg_CH1", 0)
setMCUVarUtil("SPDProtectFlg", 0)

setMCUVarUtil("CH1_cur_mode", 0)
setMCUVarUtil("speed_mode", 0)

setMCUVarUtil("CH1_ext_fcn", 1)

setMCUVarUtil("CH1_angle_mode", 0)
setMCUVarUtil("targetThetaE_CH1", 0)
setMCUVarUtil("channel_mode", 1)
setMCUVarUtil("CH1_cur_mode", 2)
setMCUVarUtil("speed_mode", 1)

In [ ]:
# speed loop run prepare
setMCUVarUtil("IProtectFlg_CH1", 0)
setMCUVarUtil("SPDProtectFlg", 0)

setMCUVarUtil("CH1_cur_mode", 0)
setMCUVarUtil("speed_mode", 0)

setMCUVarUtil("CH1_ext_fcn", 7)

setMCUVarUtil("CH1_angle_mode", 3)
setMCUVarUtil("channel_mode", 1)
setMCUVarUtil("torque_mode", 0)

setMCUVarUtil("targetRampVal", 0)
setMCUVarUtil("targetRampGrad", 500)

setMCUVarUtil("CH1_cur_mode", 2)
setMCUVarUtil("speed_mode", 5)


setMCUVarUtil("targetRampVal", 600)

In [ ]:
# speed loop run prepare2
setMCUVarUtil("IProtectFlg_CH2", 0)
setMCUVarUtil("SPDProtectFlg", 0)

setMCUVarUtil("CH2_cur_mode", 0)
setMCUVarUtil("speed_mode2", 0)

setMCUVarUtil("CH2_ext_fcn", 7)

setMCUVarUtil("CH2_angle_mode", 3)
setMCUVarUtil("channel_mode2", 1)
setMCUVarUtil("torque_mode2", 0)

setMCUVarUtil("targetRampVal2", 0)
setMCUVarUtil("targetRampGrad2", 500)

setMCUVarUtil("CH2_cur_mode", 2)
setMCUVarUtil("speed_mode2", 5)


setMCUVarUtil("targetRampVal2", 900)

In [ ]:
# 辨识算法测试

setMCUVarUtil("LMSanfThetaM.mu", .2e-4)

setMCUVarUtil("CH1_ext_fcn", 7)
setMCUVarUtil("CH1_angle_mode", 3)

setMCUVarUtil("CH1_cur_mode", 3)


# 本底
# setMCUVarUtil("CH1_angle_mode2", 4)
# 加注入
setMCUVarUtil("CH1_angle_mode2", 4+1)

setMCUVarUtil("targetRampVal2", 600)

time.sleep(15)

# 手动触发cdb，按之前的配置
setTrigConf()

time.sleep(1)

setMCUVarUtil("CH1_angle_mode2", 0)
setMCUVarUtil("targetRampVal2", 0)

In [ ]:
# kahan计算的二阶滤波器截止频率设定

ctrl_ts = 1.0/30e3
filt_freq = 5

A = 1.0 - math.exp(-2.0 * math.pi * filt_freq * ctrl_ts)

setMCUVarUtil("CH1_IdFilt_2.A", A)
setMCUVarUtil("CH1_IqFilt_2.A", A)

In [ ]:
w_temp = np.zeros(6)

w_temp[0] = readMCUVarUtil("LMSanfThetaM.W[0]")
w_temp[1] = readMCUVarUtil("LMSanfThetaM.W[1]")
w_temp[2] = readMCUVarUtil("LMSanfThetaM.W[2]")
w_temp[3] = readMCUVarUtil("LMSanfThetaM.W[3]")
w_temp[4] = readMCUVarUtil("LMSanfThetaM.W[4]")
w_temp[5] = readMCUVarUtil("LMSanfThetaM.W[5]")

print(w_temp)

In [ ]:
w0 = w_temp

In [ ]:
w1 = w_temp

In [ ]:
w1-w0

In [ ]:
# 死区补偿参数调整

db_Ithd_1 = 2
db_cmp_tick = 200
db_cmp_vds = 1.2

setMCUVarUtil("db_Ithd_1", db_Ithd_1)
setMCUVarUtil("db_cmp_tick", db_cmp_tick)
setMCUVarUtil("db_cmp_vds", db_cmp_vds)
print(readMCUVarUtil("db_Ithd_1"))
print(readMCUVarUtil("db_cmp_tick"))
print(readMCUVarUtil("db_cmp_vds"))

In [ ]:
# 旋转电角度
tyj_thetaE = 0.0
setMCUVarUtil("targetThetaE_CH2", tyj_thetaE)

In [ ]:
# oeca run prepare

setMCUVarUtil("IProtectFlg_CH1", 0)
setMCUVarUtil("SPDProtectFlg", 0)

setMCUVarUtil("CH1_cur_mode", 0)
setMCUVarUtil("speed_mode", 0)

setMCUVarUtil("CH1_angle_mode", 3)
setMCUVarUtil("channel_mode", 1)
setMCUVarUtil("CH1_cur_mode", 2)

setMCUVarUtil("targetRampGrad", 400)

In [ ]:
# 集中修改ADRC控制器参数并回读显示
ADRC_mode = 0

ADRC_gamma = 0.00065
ADRC_c_u1 = 15
ADRC_c_u2 = 200
ADRC_beta1 = 2500
ADRC_beta2 = (ADRC_beta1/2)**2*0.45
ADRC_SMCk = 0

ADRC_TeMax = 8.0
ADRC_TeMin = -22.0

b0_k = 1.2

setMCUVarUtil("UdcAdrc.para_gamma", ADRC_gamma)
setMCUVarUtil("UdcAdrc.para_cu1", ADRC_c_u1)
setMCUVarUtil("UdcAdrc.para_cu2", ADRC_c_u2)
setMCUVarUtil("UdcAdrc.para_beta1", ADRC_beta1)
setMCUVarUtil("UdcAdrc.para_beta2", ADRC_beta2)
setMCUVarUtil("UdcAdrc.para_SMCk", ADRC_SMCk)
setMCUVarUtil("ADRC_mode", ADRC_mode)
setMCUVarUtil("UdcAdrc.para_TeMax", ADRC_TeMax)
setMCUVarUtil("UdcAdrc.para_TeMin", ADRC_TeMin)
setMCUVarUtil("UdcAdrc.b0_k", b0_k)
print(readMCUVarUtil("UdcAdrc.para_gamma"))
print(readMCUVarUtil("UdcAdrc.para_cu1"))
print(readMCUVarUtil("UdcAdrc.para_cu2"))
print(readMCUVarUtil("UdcAdrc.para_beta1"))
print(readMCUVarUtil("UdcAdrc.para_beta2"))
print(readMCUVarUtil("UdcAdrc.para_SMCk"))
print(readMCUVarUtil("ADRC_mode"))
print(readMCUVarUtil("UdcAdrc.para_TeMax"))
print(readMCUVarUtil("UdcAdrc.para_TeMin"))
print(readMCUVarUtil("UdcAdrc.b0_k"))

In [17]:
# 磁链数据标定
import numpy as np

# 标定参数设定
DIM_N = 16
MAX_I = 10
step = MAX_I/(DIM_N-1)

spdRPM = 1200
p0 = 4
omegaE = spdRPM/60.0*p0*2.0*math.pi

delaytimeH = 0.5
delaytimeL = 0.2

IdR1 = MAX_I*0.25
IdR2 = MAX_I*0.5

# 这里访问变量时是 mat[y][x] 与C语言实现不同
testId = -np.arange(0, MAX_I+step, step)
testIq = np.arange(0, MAX_I+step, step)

IdX, IqY = np.meshgrid(testId, testIq)

# 仅Id时，用来评估电阻(两次测量减弱死区影响？)
UdR1 = np.zeros(IdX.shape)
UqR1 = np.zeros(IdX.shape)
UdR2 = np.zeros(IdX.shape)
UqR2 = np.zeros(IdX.shape)
# 测量值
Ud1 = np.zeros(IdX.shape)
Uq1 = np.zeros(IdX.shape)

In [ ]:
# 状态准备（原在转速闭环状态下）
setMCUVarUtil("speed_mode", 0)
setMCUVarUtil("channel_mode", 2)
setMCUVarUtil("torque_mode", 1)
setMCUVarUtil("targetThetaMTPA", 0)
setMCUVarUtil("targetRampVal", 0)
setMCUVarUtil("targetRampGrad", MAX_I*50)
setMCUVarUtil("speed_mode", 1)

# 设置测功机（仅限自制平台）
setMCUVarUtil("targetRampVal2", spdRPM)

# 稳定后获得零电流时的电压参考值
time.sleep(2)
Ud0 = readMCUVarUtil("CH1_UTarFiltd.C1")
Uq0 = readMCUVarUtil("CH1_UTarFiltq.C1")

In [ ]:
# 正式进行

# 采用固定Id 调整Iq的方法
# 电流总是先小后大，预期得到慢慢增大的Rs估计值
for ii in range(IdX.shape[1]):
    for jj in range(IdX.shape[0]):
        tarId = IdX[jj][ii]
        tarIq = IqY[jj][ii]
        tarIs = math.sqrt(tarId**2 + tarIq**2)
        theta = math.atan2(tarIq, tarId)/math.pi/2
        theta = theta-math.floor(theta)
        print(f"set to {tarId},{tarIq}")
        print(f"as {tarIs},{theta}")
        # 至工况点
        setMCUVarUtil("targetThetaMTPA", theta)
        setMCUVarUtil("targetRampVal", tarIs)
        # 等待稳定
        time.sleep(delaytimeH)
        # 记录数据
        Ud1[jj][ii] = readMCUVarUtil("CH1_UTarFiltd.C1")
        Uq1[jj][ii] = readMCUVarUtil("CH1_UTarFiltq.C1")
        # 至电阻测量工况点1
        tarId = IdR1
        tarIq = 0
        tarIs = math.sqrt(tarId**2 + tarIq**2)
        theta = math.atan2(tarIq, tarId)/math.pi/2
        theta = theta-math.floor(theta)
        setMCUVarUtil("targetThetaMTPA", theta)
        setMCUVarUtil("targetRampVal", tarIs)
        # 等待稳定
        time.sleep(delaytimeH)
        # 记录数据
        UdR1[jj][ii] = readMCUVarUtil("CH1_UTarFiltd.C1")
        UqR1[jj][ii] = readMCUVarUtil("CH1_UTarFiltq.C1")
        # 至电阻测量工况点2
        tarId = IdR2
        tarIq = 0
        tarIs = math.sqrt(tarId**2 + tarIq**2)
        theta = math.atan2(tarIq, tarId)/math.pi/2
        theta = theta-math.floor(theta)
        setMCUVarUtil("targetThetaMTPA", theta)
        setMCUVarUtil("targetRampVal", tarIs)
        # 等待稳定，这次没有转矩\转速波动，只有电流波动，可以少等会
        time.sleep(delaytimeL)
        # 记录数据
        UdR2[jj][ii] = readMCUVarUtil("CH1_UTarFiltd.C1")
        UqR2[jj][ii] = readMCUVarUtil("CH1_UTarFiltq.C1")

# 测试完成，降电流
setMCUVarUtil("targetRampVal", 0)

In [26]:
# 保存为matlab格式
import scipy.io

# 将 NumPy 数组保存为 .mat 文件
scipy.io.savemat('faifDataRaw.mat', {'IdX': IdX, 'IqY': IqY, 'Ud0': Ud0, 'Uq0': Uq0, 'Ud1': Ud1, 'Uq1': Uq1,
                                     'UdR1': UdR1, 'UqR1': UqR1, 'UdR2': UdR2, 'UqR2': UqR2,
                                     'omegaE': omegaE, 'IdR1': IdR1, 'IdR2': IdR2, 'step': step})


In [22]:
# 临时生成测试数据
import random

Rs = 0.2
Ld = 0.13e-3
Lq = 0.13e-3
faif = 0.0075

Ud0 = 0
Uq0 = 0

UqR1 = np.zeros(IdX.shape)*Rs*IdR1+np.random.uniform(-0.05, 0.05, IdX.shape)
UqR2 = np.zeros(IdX.shape)*Rs*IdR1+np.random.uniform(-0.05, 0.05, IdX.shape)

UdR1 = np.ones(IdX.shape)*Rs*IdR1+np.random.uniform(-0.05, 0.05, IdX.shape)
UdR2 = np.ones(IdX.shape)*Rs*IdR2+np.random.uniform(-0.05, 0.05, IdX.shape)

for ii in range(IdX.shape[1]):
    for jj in range(IdX.shape[0]):
        tarId = IdX[jj][ii]
        tarIq = IqY[jj][ii]
        Ud1[jj][ii] = Rs*tarId+omegaE*Lq*tarIq+random.uniform(-0.05, 0.05)
        Uq1[jj][ii] = Rs*tarIq-omegaE * \
            (Ld*tarId+faif)+random.uniform(-0.05, 0.05)
